In [1]:
import json
import time
import requests
import hmac
import hashlib
import time
import pandas as pd
import cbpro
from datetime import datetime, timedelta
from joblib import load
from urllib3.exceptions import ProtocolError, ConnectionError

In [2]:
with open('../coinbase-secrets.json', 'r') as jfile:
    secrets = json.loads(jfile.read())
    jfile.close()

## Coinbase Pro

In [3]:
public_client = cbpro.PublicClient()

In [4]:
public_client.get_product_ticker(product_id='ETH-USD')

{'trade_id': 144154438,
 'price': '2972.7',
 'size': '0.00451707',
 'time': '2021-08-09T00:18:54.456259Z',
 'bid': '2972.36',
 'ask': '2972.7',
 'volume': '236479.45171938'}

In [5]:
time_data = public_client.get_time()
end = '2021-06-28T23:59:59.999Z'
start = '2021-06-28T23:00:00.000Z'

In [6]:
columns = ['timestamp', 'low', 'high', 'open', 'close', 'volume']

In [7]:
historic_data = public_client.get_product_historic_rates(
    product_id = 'ETH-USD', 
    start=str(start), 
    end=str(end), 
    granularity=3600
)
historic_data = pd.DataFrame(historic_data, columns=columns)
historic_data['timestamp'] = pd.to_datetime(historic_data['timestamp'], unit='s')
historic_data

,timestamp,low,high,open,close,volume
0,2021-06-28 23:00:00,2062.35,2097.5,2097.5,2083.79,9784.226516


In [8]:
end = time_data['iso']
print(end)

2021-08-09T00:18:55.949Z


In [9]:
end_datetime = datetime.fromtimestamp(time_data['epoch'])
start_datetime = end_datetime - timedelta(hours=1)
end_iso = end_datetime.isoformat()
start_iso = start_datetime.isoformat()

In [10]:
historic_data = public_client.get_product_historic_rates(
    product_id = 'ETH-USD', 
    start=start_iso, 
    end=end_iso, 
    granularity=3600
)
historic_data = pd.DataFrame(historic_data, columns=columns)
historic_data['timestamp'] = pd.to_datetime(historic_data['timestamp'], unit='s')
historic_data

,timestamp,low,high,open,close,volume
0,2021-08-08 20:00:00,2963,3025.04,2985.43,2992.08,13388.650839


In [11]:
model = load('../models/linear_model.joblib')
batch = historic_data[['open', 'high', 'low', 'close', 'volume']].values
prediction = model.predict(batch)
prediction

array([2059.55241498])

In [12]:
import pickle
import time
from datetime import datetime, timedelta

import cbpro
from urllib3.exceptions import ConnectionError, ProtocolError


class AssetTrader(object):
    def __init__(
        self,
        asset: str,
        api_secret: str,
        api_key: str,
        passphrase: str,
        pickle_file: str,
        use_sandbox: bool = True,
    ):
        self.asset = asset
        self.api_secret = api_secret
        self.public_client = cbpro.PublicClient()
        api_url = ""
        if use_sandbox:
            api_url = "https://api-public.sandbox.pro.coinbase.com"
        else:
            api_url = "https://api.pro.coinbase.com"

        self.private_client = cbpro.AuthenticatedClient(
            key=api_key,
            b64secret=api_secret.encode(),
            passphrase=passphrase,
            api_url=api_url,
        )
        self.accounts = self.private_client.get_accounts()
        try:
            for account in self.accounts:
                if account["currency"] == "USD":
                    self.usd_wallet = account["id"]
                elif account["currency"] == self.asset.split("-")[0]:
                    self.asset_wallet = account["id"]
        except TypeError as e:
            print(f'Error retrieving account information: {self.accounts}\n{e}')

        with open(pickle_file, 'rb') as pfile:
            self.model = pickle.load(pfile)
            pfile.close()

    def _get_start_end_iso_times(self, hours: int = 1):
        """
        From the current iso formatted timestamp, this generates
        a start and end datetime that are 1 hour apart.

        :returns: (tuple) Contains (start, end) datetimes.
        """

        time_data = self.public_client.get_time()

        end_datetime = datetime.fromtimestamp(time_data["epoch"])
        start_datetime = end_datetime - timedelta(hours=hours)

        end_iso = end_datetime.isoformat()
        start_iso = start_datetime.isoformat()
        return (start_iso, end_iso)

    def get_asset_details_last_hour(
        self, start: str, end: str, granularity: int = 3600
    ):
        """
        Retrieves hourly open, high, low, close, and volume for the given asset
        over the time range on start to end broken into granularity seconds.

        :param start: (str) ISO-8601 formatted timestamp.
        :param end: (str) ISO-8601 formatted timestamp.
        :param granularity: (int) Number of seconds per interval between start and end.
        :returns: (np.array) Array containing the detailed asset price data.
        """
        try:
            historic_data = self.public_client.get_product_historic_rates(
                product_id=self.asset, start=start, end=end, granularity=granularity
            )
        except (ProtocolError, ConnectionError):
            time.sleep(5)
            historic_data = self.public_client.get_product_historic_rates(
                product_id=self.asset, start=start, end=end, granularity=granularity
            )
        
        historic_dataset = {
            'timestamp': historic_data[0][0],
            'open': historic_data[0][1],
            'high': historic_data[0][2],
            'low': historic_data[0][3],
            'close': historic_data[0][4],
            'volume': historic_data[0][5]
        }

        return historic_dataset

    def get_account_balance(self, account_id: str):
        """Retrieves the account balance for a given account_id"""

        account_details = self.private_client.get_account(account_id)
        return float(account_details["balance"])

    def predict(
        self,
        open_: float,
        high_: float,
        low_: float,
        close_: float,
        asset_volume_: float,
    ):
        """Uses self.model to predict the close price 1-hour from now."""

        batch = [open_, high_, low_, close_, asset_volume_]
        model_prediction = self.model.predict([batch])
        return model_prediction

    def place_buy_order(self, amount: float):
        """
        Checks to see if the amount to buy is greater than USD funds
        available, if so amount is set to the USD funds. Places a buy
        order.
        """

        usd_balance = self.get_account_balance(self.usd_wallet)

        if amount > usd_balance:
            amount = usd_balance

        buy_order_response = self.private_client.place_market_order(
            product_id=self.asset, side="buy", funds=str(amount)
        )
        return buy_order_response

    def place_sell_order(self, amount: float):
        """
        Checks to see if the amount to sell is greater than asset funds
        available, if so amount is set to the asset funds. Places a sell
        order.
        """

        asset_balance = self.get_account_balance(self.asset_wallet)

        if amount > asset_balance:
            amount = asset_balance

        sell_order_response = self.private_client.place_market_order(
            product_id=self.asset, side="sell", funds=str(amount)
        )
        return sell_order_response

    def trading_strategy(
        self,
        model_prediction: float,
        threshold_to_act: float,
        current_close_price: float,
        percent_of_total_money_to_move: float,
        total_money_in_usd: float,
    ):
        """
        Determines whether to buy, sell, or do nothing as well as an amount.
        If action is buy, amount is the amount of USD to spend.
        If action is sell, amount is the amount of Asset to sell.
        If action is do_nothing, amount is 0.0.
        """

        # threshold_to_act = validation_metrics['mae'] / 3
        action = "do_nothing"
        if abs(model_prediction - current_close_price) > threshold_to_act:
            if model_prediction - current_close_price > 0:
                action = "buy"
            else:
                action = "sell"

        amount_of_usd_to_exchange = percent_of_total_money_to_move * total_money_in_usd
        amount_of_asset_to_exchange = amount_of_usd_to_exchange / current_close_price

        if action == "buy":
            amount_to_exchange = amount_of_usd_to_exchange
        elif action == "sell":
            amount_to_exchange = amount_of_asset_to_exchange
        else:
            amount_to_exchange = 0.0

        return (action, amount_to_exchange)


In [25]:
asset_trader = AssetTrader(
    asset='ETH-USD', 
    api_secret=secrets['SECRET'], 
    api_key=secrets['KEY'], 
    passphrase=secrets['PASSPHRASE'],
    use_sandbox=False,
    pickle_file='../models/linearregressor-20210808T202036/linearregressor-20210808T202036.pickle'
)

In [26]:
start, end = asset_trader._get_start_end_iso_times()
last_hour = asset_trader.get_asset_details_last_hour(start, end)
last_hour

{'timestamp': 1628452800,
 'open': 2963,
 'high': 3025.04,
 'low': 2985.43,
 'close': 2992.08,
 'volume': 13388.65083898}

In [27]:
asset_trader.get_account_balance(asset_trader.usd_wallet)

100.0

In [28]:
asset_trader.get_account_balance(asset_trader.asset_wallet)

0.05173289

In [17]:
response = asset_trader.place_order(
    account_id=asset_trader.usd_wallet, 
    amount=10.0,
    order_type='buy'
)

In [27]:
orders = asset_trader.private_client.get_order(response['id'])
orders

{'id': 'a89c6a4d-3565-4254-8472-217fe4c82b77',
 'product_id': 'BTC-USD',
 'profile_id': '1d2bb766-5156-4029-ade7-4d4d94a50ce3',
 'side': 'buy',
 'funds': '9.9502487500000000',
 'specified_funds': '10.0000000000000000',
 'type': 'market',
 'post_only': False,
 'created_at': '2021-07-02T12:10:00.556232Z',
 'done_at': '2021-07-02T12:10:00.561Z',
 'done_reason': 'filled',
 'fill_fees': '0.0497502532800000',
 'filled_size': '0.00030121',
 'executed_value': '9.9500506560000000',
 'status': 'done',
 'settled': True}